## TLE Parser
* https://pypi.org/project/ephem/ 
* https://federicostra.github.io/tletools/ 

In [85]:
# from IPython.display import Image, HTML
import numpy as np
import pandas as pd
import geopy.distance
import math
#import time
import datetime
import ephem
from math import degrees

import warnings

warnings.filterwarnings("ignore")           # Suppress Warning

In [86]:
WorkingFolder = "/Users/cv0361/Desktop/TechChallenge/Data/"

In [87]:
# # Sample Success Parse
# line1 = "ISS (ZARYA)"
# line2 = "1 25544U 98067A   03097.78853147  .00021906  00000-0  28403-3 0  8652"
# line3 = "2 25544  51.6361  13.7980 0004256  35.6671  59.2566 15.58778559250029"

# iss = ephem.readtle(line1, line2, line3)

# print("SatelliteId: {}\tTLE Epoch: {}".format(iss.catalog_number, iss._epoch))

# iss.compute(iss._epoch)
# print("Lat: {}\tLong: {}".format(degrees(iss.sublat), degrees(iss.sublong)))

# iss.compute('2003/4/7 19:00')
# print("Lat: {}\tLong: {}".format(degrees(iss.sublat), degrees(iss.sublong)))


In [88]:
# # Data causing parsing error
# line1 = "NULL"
# line2 = "1 41857U 16068A   17001.92200105 -.00000043 +00000-0 +00000-0 0 0173"
# line3 = "2 41857 098.5098 004.1411 0012813 111.3144 248.9419 14.3270115000730"

# iss = ephem.readtle(line1, line2, line3)


In [89]:
filename = WorkingFolder + "TLE/tle2013.txt"

with open(filename, 'r') as f:
    Lines = f.readlines()

print("Total lines:", len(Lines))

Total lines: 5924618


In [90]:
Header = ['SatNum', 'Class', 'InternDes', 'Epoch', 'Ballistic', 'MeanMotion', 'DragTerm', 'Ephemeris', 'ElementNum', 
          'Inclination', 'RightAsc', 'Eccentricity', 'ArgPerigee', 'MeanAno', 'MeanMot', 'Lat', 'Lon', 'EpochDate', 'errTLE']

def parseTLE(line1, line2, line3):
    x = line2.split()
    SatNum = x[1][:-1]
    Classi = x[1][-1:]
    return [SatNum, Classi] + x[2:] + line3.split()[2:8]

# line1 = "ISS (ZARYA)"
# line2 = "1 25544U 98067A   03097.78853147  .00021906  00000-0  28403-3 0  8652"
# line3 = "2 25544  51.6361  13.7980 0004256  35.6671  59.2566 15.58778559250029"
# parseTLE(line1, line2, line3)

In [91]:
count = 0
line1 = "NULL" 
data_array = list()

try:
    for line in Lines: 
        count += 1

        if count%2:
            line2 = line.strip().replace("\\", "")

        else:
            line3 = line.strip()

#             if "41857U" not in line2: continue   # Filter by specific satellite

            try:
                iss = ephem.readtle(line1, line2, line3)
            except:
#                 print("*********** Parsing Error")
                data_array.append(parseTLE(line1, line2, line3) + [None, None, '', 1])
                continue

            iss.compute(iss._epoch)
            lat = round(degrees(iss.sublat), 6)
            lon = round(degrees(iss.sublong), 6)
            
            data_array.append(parseTLE(line1, line2, line3) + [lat, lon, str(iss._epoch), 0])
            
#             print(line2)
#             print(line3)
#             print("SatId: {}\tEpoch: {}\tLat: {}\tLong: {}".format(iss.catalog_number, iss._epoch, lat, lon))
#             print()

#         if count > 7: break   # Get off early
except:
    pass

print("Line:", count)

df = pd.DataFrame(data_array, columns=Header)
df.head()

Line: 407034


,SatNum,Class,InternDes,Epoch,Ballistic,MeanMotion,DragTerm,Ephemeris,ElementNum,Inclination,RightAsc,Eccentricity,ArgPerigee,MeanAno,MeanMot,Lat,Lon,EpochDate,errTLE
0,16615,U,86019C,12366.73111925,.00000103,00000-0,56613-4,0,02,098.8319,040.3042,0012822,141.1725,327.9666,14.31005531406538,69.027586,-119.510593,2012/12/31 17:32:49,0
1,13769,U,83002J,12366.69743804,-.00000062,00000-0,-43902-3,0,2789,074.0059,046.9329,0136369,207.9895,302.8637,12.21309982335584,29.092754,-133.835535,2012/12/31 16:44:19,0
2,20791,U,90081D,12366.77263477,.00000114,00000-0,12150-3,0,4496,099.1974,056.6406,0053484,180.0160,285.8788,13.92464973134683,72.174957,-111.834075,2012/12/31 18:32:36,0
3,27847,U,03031H,12366.99213746,.00000118,00000-0,74070-4,0,4564,098.6898,012.7356,0009343,354.3909,005.7160,14.21348280493057,-0.000214,-85.232886,2012/12/31 23:48:41,0
4,26188,U,99057CA,12366.70510263,.00004245,00000-0,76218-3,0,7302,098.8739,034.0631,0051130,133.9335,294.5412,14.66456360678037,66.265478,18.896557,2012/12/31 16:55:21,0


In [92]:
df.loc[df.errTLE == 1].head()

,SatNum,Class,InternDes,Epoch,Ballistic,MeanMotion,DragTerm,Ephemeris,ElementNum,Inclination,RightAsc,Eccentricity,ArgPerigee,MeanAno,MeanMot,Lat,Lon,EpochDate,errTLE


In [93]:
# Output result to csv
df.to_csv(WorkingFolder + "TLE/tle2013.csv", index=False)